In [ ]:
import subprocess

# Install kaggle package
subprocess.run(['pip', 'install', 'kaggle'])

# Import necessary libraries
from kaggle.api.kaggle_api_extended import KaggleApi

# Authenticate with your Kaggle credentials
api = KaggleApi()
api.authenticate()  # Make sure to authenticate with your Kaggle credentials

# Download the Road Vehicle Images Dataset
api.dataset_download_files('sakshamjn/vehicle-detection-8-classes-object-detection', path = 'kaggle-dataset/'
, unzip=True)


Dataset URL: https://www.kaggle.com/datasets/sakshamjn/vehicle-detection-8-classes-object-detection


In [ ]:
import subprocess
import os
import zipfile

# Install kaggle package
subprocess.run(['pip', 'install', 'kaggle'])

# Import necessary libraries
from kaggle.api.kaggle_api_extended import KaggleApi

# Authenticate with your Kaggle credentials
api = KaggleApi()
api.authenticate()

# Define the download path
download_path = 'C:/Users/mahnoor/Documents/Python Scripts/kaggle-dataset/'

# Download the Road Vehicle Images Dataset
api.dataset_download_files('sakshamjn/vehicle-detection-8-classes-object-detection', path=download_path, unzip=True)

# Unzip the dataset if not already unzipped
zip_file_path = os.path.join(download_path, 'vehicle-detection-8-classes-object-detection.zip')

if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(download_path)
    os.remove(zip_file_path)  # Remove the zip file after extracting

print("Dataset downloaded and extracted successfully.")


Dataset URL: https://www.kaggle.com/datasets/sakshamjn/vehicle-detection-8-classes-object-detection


# Model Training

In [8]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Define dataset paths
images_path = 'C:/Users/mahnoor/Documents/Python Scripts/kaggle-dataset/train/images'
labels_path = 'C:/Users/mahnoor/Documents/Python Scripts/kaggle-dataset/train/labels.npy'

# Check if the dataset paths exist
if not os.path.exists(images_path):
    raise FileNotFoundError(f"The specified images path does not exist: {images_path}")

if not os.path.exists(labels_path):
    raise FileNotFoundError(f"The specified labels path does not exist: {labels_path}")

# Load labels
labels = np.load(labels_path, allow_pickle=True)

# Print the shape and a few entries of the labels array to understand its structure
print(f"Labels shape: {labels.shape}")
print(f"First few labels: {labels[:5]}")

# Extract only the class labels from the complex label structures
class_labels = []
for label in labels:
    if len(label) > 0:
        class_labels.append(label[0][0])
    else:
        class_labels.append(-1)  # Placeholder for missing labels

class_labels = np.array(class_labels)

# Load images and corresponding labels
images = []
filtered_labels = []
image_files = sorted([f for f in os.listdir(images_path) if f.endswith(('png', 'jpg', 'jpeg'))])

for idx, img_name in enumerate(image_files):
    img_path = os.path.join(images_path, img_name)
    img = cv2.imread(img_path)
    if img is not None and class_labels[idx] != -1:
        img = cv2.resize(img, (128, 128))  # Resize images to 128x128
        images.append(img)
        filtered_labels.append(class_labels[idx])
    else:
        print(f"Warning: Skipping image {img_path} due to missing or invalid label")

images = np.array(images)
filtered_labels = np.array(filtered_labels)

# Ensure the number of images matches the number of filtered labels
if len(images) != len(filtered_labels):
    raise ValueError(f"Number of images ({len(images)}) does not match number of filtered labels ({len(filtered_labels)})")

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, filtered_labels, test_size=0.2, random_state=42)

# Normalize the images
X_train = X_train / 255.0
X_val = X_val / 255.0

# Convert labels to one-hot encoding
num_classes = len(np.unique(filtered_labels))
y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

# Save the model
model.save('vehicle_detection_model.h5')

print("Model trained and saved successfully.")


Labels shape: (8218,)
First few labels: [array([[3.      , 0.509804, 0.411765, 0.107843, 0.245098],
        [2.      , 0.210784, 0.616422, 0.127451, 0.232843]], dtype=float32)
 array([[0.     , 0.50375, 0.405  , 0.0975 , 0.245  ],
        [1.     , 0.19   , 0.59875, 0.12   , 0.2325 ],
        [2.     , 0.62125, 0.61   , 0.0575 , 0.125  ]], dtype=float32)
 array([[3.      , 0.516169, 0.396766, 0.10199 , 0.246269],
        [2.      , 0.205224, 0.594527, 0.121891, 0.233831],
        [4.      , 0.631841, 0.597015, 0.059701, 0.124378]], dtype=float32)
 array([[2.      , 0.141827, 0.539663, 0.110577, 0.165865],
        [2.      , 0.360577, 0.278846, 0.0625  , 0.086538],
        [2.      , 0.634615, 0.649038, 0.0625  , 0.129808]], dtype=float32)
 array([[2.      , 0.140625, 0.528846, 0.117788, 0.168269],
        [2.      , 0.366587, 0.274038, 0.064904, 0.086538],
        [2.      , 0.628606, 0.653846, 0.064904, 0.129808]], dtype=float32)]



Epoch 1/10


205/205 [=============================

C:\Users\mahnoor\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model trained and saved successfully.


# Transfer Learning (Training on pre-trained model)      ResNet

In [10]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

# Define dataset paths
images_path = 'C:/Users/mahnoor/Documents/Python Scripts/coco2017/train2017/images'
labels_path = 'C:/Users/mahnoor/Documents/Python Scripts/coco2017/train/labels.npy'

# Check if the dataset paths exist
if not os.path.exists(images_path):
    raise FileNotFoundError(f"The specified images path does not exist: {images_path}")

if not os.path.exists(labels_path):
    raise FileNotFoundError(f"The specified labels path does not exist: {labels_path}")

# Load labels
labels = np.load(labels_path, allow_pickle=True)

# Print the shape and a few entries of the labels array to understand its structure
print(f"Labels shape: {labels.shape}")
print(f"First few labels: {labels[:5]}")

# Extract only the class labels from the complex label structures
class_labels = []
for label in labels:
    if len(label) > 0:
        class_labels.append(label[0][0])
    else:
        class_labels.append(-1)  # Placeholder for missing labels

class_labels = np.array(class_labels)

# Load images and corresponding labels
images = []
filtered_labels = []
image_files = sorted([f for f in os.listdir(images_path) if f.endswith(('png', 'jpg', 'jpeg'))])

for idx, img_name in enumerate(image_files):
    img_path = os.path.join(images_path, img_name)
    img = cv2.imread(img_path)
    if img is not None and class_labels[idx] != -1:
        img = cv2.resize(img, (128, 128))  # Resize images to 128x128
        images.append(img)
        filtered_labels.append(class_labels[idx])
    else:
        print(f"Warning: Skipping image {img_path} due to missing or invalid label")

images = np.array(images)
filtered_labels = np.array(filtered_labels)

# Print the shape and a few entries of the filtered datasets
print(f"Filtered images shape: {images.shape}")
print(f"Filtered labels shape: {filtered_labels.shape}")
print(f"First few filtered labels: {filtered_labels[:5]}")

# Ensure the number of images matches the number of filtered labels
if len(images) != len(filtered_labels):
    raise ValueError(f"Number of images ({len(images)}) does not match number of filtered labels ({len(filtered_labels)})")

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, filtered_labels, test_size=0.2, random_state=42)

# Normalize the images
X_train = X_train / 255.0
X_val = X_val / 255.0

# Convert labels to one-hot encoding
num_classes = len(np.unique(filtered_labels))
y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(filtered_labels), y=filtered_labels)
class_weights = dict(enumerate(class_weights))

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=50, validation_data=(X_val, y_val),
                    class_weight=class_weights, callbacks=[reduce_lr, early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

# Save the model
model.save('vehicle_detection_model_resnet50_improved.h5')

print("Model trained and saved successfully.")


Labels shape: (8218,)
First few labels: [array([[3.      , 0.509804, 0.411765, 0.107843, 0.245098],
        [2.      , 0.210784, 0.616422, 0.127451, 0.232843]], dtype=float32)
 array([[0.     , 0.50375, 0.405  , 0.0975 , 0.245  ],
        [1.     , 0.19   , 0.59875, 0.12   , 0.2325 ],
        [2.     , 0.62125, 0.61   , 0.0575 , 0.125  ]], dtype=float32)
 array([[3.      , 0.516169, 0.396766, 0.10199 , 0.246269],
        [2.      , 0.205224, 0.594527, 0.121891, 0.233831],
        [4.      , 0.631841, 0.597015, 0.059701, 0.124378]], dtype=float32)
 array([[2.      , 0.141827, 0.539663, 0.110577, 0.165865],
        [2.      , 0.360577, 0.278846, 0.0625  , 0.086538],
        [2.      , 0.634615, 0.649038, 0.0625  , 0.129808]], dtype=float32)
 array([[2.      , 0.140625, 0.528846, 0.117788, 0.168269],
        [2.      , 0.366587, 0.274038, 0.064904, 0.086538],
        [2.      , 0.628606, 0.653846, 0.064904, 0.129808]], dtype=float32)]
Filtered images shape: (8200, 128, 128, 3)
Filtered la

C:\Users\mahnoor\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model trained and saved successfully.


In [8]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('vehicle_detection_model.h5')

# Initialize video capture (0 for default camera, or provide video file path)
cap = cv2.VideoCapture(0)

# Parameters
input_size = (128, 128)  # Input size for the model
stationary_threshold = 15  # Time in seconds (2 minutes)
stationary_objects = {}  # Dictionary to track stationary objects

def preprocess_frame(frame):
    """Preprocess frame for model prediction."""
    img = cv2.resize(frame, input_size)
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)
    return img

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get frame dimensions
    height, width = frame.shape[:2]

    # Preprocess the frame for prediction
    img = preprocess_frame(frame)
    
    # Predict using the model
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Only consider vehicle classes (assuming vehicle classes are labeled from 1 to N)
    if predicted_class in [0, 1, 2, 3, 4, 5, 6, 7]:  # Adjust based on your class indices
        # For simplicity, assume the entire frame is the bounding box
        bbox = (0, 0, width, height)  # You would normally get this from your detection model
        
        # Track stationary objects
        if predicted_class not in stationary_objects:
            stationary_objects[predicted_class] = {'start_time': time.time(), 'bbox': bbox}
        else:
            elapsed_time = time.time() - stationary_objects[predicted_class]['start_time']
            if elapsed_time > stationary_threshold:
                # Draw red bounding box around the vehicle
                (x, y, w, h) = stationary_objects[predicted_class]['bbox']
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            else:
                stationary_objects[predicted_class]['bbox'] = bbox
    else:
        # Reset the timer if the object moves
        if predicted_class in stationary_objects:
            del stationary_objects[predicted_class]

    # Display the frame with bounding boxes
    cv2.imshow('Static Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 45ms/step


In [15]:
import cv2
import numpy as np

# Path to the static car image
car_image_path = r"C:\Users\mahnoor\Documents\Python Scripts\static_car.jpg"

# Load the car image
car_image = cv2.imread(car_image_path)

# Define video parameters
frame_width = car_image.shape[1]
frame_height = car_image.shape[0]
fps = 30  # Frames per second
duration = 120  # Duration in seconds
total_frames = fps * duration

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('static_car_video.avi', fourcc, fps, (frame_width, frame_height))

# Write the same frame multiple times to create a video
for _ in range(total_frames):
    out.write(car_image)

# Release the video writer object
out.release()

print("Video created successfully.")


Video created successfully.


In [1]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('vehicle_detection_model.h5')

# Initialize video capture with the generated video file
video_path = 'static_car_video.avi'
cap = cv2.VideoCapture(video_path)

# Parameters
input_size = (128, 128)  # Input size for the model
stationary_threshold = 120  # Time in seconds (2 minutes)
stationary_objects = {}  # Dictionary to track stationary objects
blink_flag = True  # Flag to toggle the blinking effect
blink_interval = 10  # Interval for blinking effect (in frames)

def preprocess_frame(frame):
    """Preprocess frame for model prediction."""
    img = cv2.resize(frame, input_size)
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)
    return img

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get frame dimensions
    height, width = frame.shape[:2]

    # Preprocess the frame for prediction
    img = preprocess_frame(frame)
    
    # Predict using the model
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Only consider vehicle classes (assuming vehicle classes are labeled from 1 to N)
    if predicted_class in [0, 1, 2, 3, 4, 5, 6, 7]:  # Adjust based on your class indices
        # For simplicity, assume the entire frame is the bounding box
        bbox = (0, 0, width, height)  # You would normally get this from your detection model
        
        # Track stationary objects
        if predicted_class not in stationary_objects:
            stationary_objects[predicted_class] = {'start_time': time.time(), 'bbox': bbox}
        else:
            elapsed_time = time.time() - stationary_objects[predicted_class]['start_time']
            if elapsed_time > stationary_threshold:
                # Draw red bounding box around the vehicle
                (x, y, w, h) = stationary_objects[predicted_class]['bbox']
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            else:
                stationary_objects[predicted_class]['bbox'] = bbox
    else:
        # Reset the timer if the object moves
        if predicted_class in stationary_objects:
            del stationary_objects[predicted_class]

    # Display the frame with bounding boxes
    cv2.imshow('Static Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()





1/1 [==============================] - 0s 47ms/step


In [7]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('vehicle_detection_model_resnet50_improved.h5')

# Initialize video capture (0 for default camera, or provide video file path)
cap = cv2.VideoCapture(0)

# Parameters
input_size = (128, 128)  # Input size for the model
stationary_threshold = 15  # Time in seconds
stationary_objects = {}  # Dictionary to track stationary objects

def preprocess_frame(frame):
    """Preprocess frame for model prediction."""
    img = cv2.resize(frame, input_size)
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)
    return img

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get frame dimensions
    height, width = frame.shape[:2]

    # Preprocess the frame for prediction
    img = preprocess_frame(frame)
    
    # Predict using the model
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Only consider vehicle classes (assuming vehicle classes are labeled from 1 to N)
    if predicted_class in [0, 1, 2, 3, 4, 5, 6, 7]:  # Adjust based on your class indices
        # For simplicity, assume the entire frame is the bounding box
        # In a real scenario, use an object detection model to get actual bounding boxes
        bbox = (100, 100, 200, 200)  # Replace this with actual bounding box coordinates from detection model

        # Track stationary objects
        if predicted_class not in stationary_objects:
            stationary_objects[predicted_class] = {'start_time': time.time(), 'bbox': bbox}
        else:
            elapsed_time = time.time() - stationary_objects[predicted_class]['start_time']
            if elapsed_time > stationary_threshold:
                # Draw red bounding box around the vehicle
                (x, y, w, h) = stationary_objects[predicted_class]['bbox']
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            else:
                stationary_objects[predicted_class]['bbox'] = bbox
    else:
        # Reset the timer if the object moves
        if predicted_class in stationary_objects:
            del stationary_objects[predicted_class]

    # Display the frame with bounding boxes
    cv2.imshow('Static Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 115ms/step


In [10]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('vehicle_detection_model.h5')

# Initialize video capture (0 for default camera, or provide video file path)
cap = cv2.VideoCapture(0)

# Parameters
input_size = (128, 128)  # Input size for the model
stationary_threshold = 15  # Time in seconds (2 minutes)
stationary_objects = {}  # Dictionary to track stationary objects

def preprocess_frame(frame):
    """Preprocess frame for model prediction."""
    img = cv2.resize(frame, input_size)
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)
    return img

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame for prediction
    img = preprocess_frame(frame)
    
    # Predict using the model
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Only consider vehicle classes (assuming vehicle classes are labeled from 1 to N)
    if predicted_class in [0, 1, 2, 3, 4, 5, 6, 7]:  # Adjust based on your class indices
        # For simplicity, assume the entire frame is the bounding box
        bbox = (0, 0, frame.shape[1], frame.shape[0])  # You would normally get this from your detection model
        
        # Track stationary objects
        if predicted_class not in stationary_objects:
            stationary_objects[predicted_class] = {'start_time': time.time(), 'bbox': bbox}
        else:
            elapsed_time = time.time() - stationary_objects[predicted_class]['start_time']
            if elapsed_time > stationary_threshold:
                # Draw red bounding box around the vehicle
                (x, y, w, h) = stationary_objects[predicted_class]['bbox']
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            else:
                stationary_objects[predicted_class]['bbox'] = bbox
    else:
        # Remove non-stationary or moved vehicles
        stationary_objects = {k: v for k, v in stationary_objects.items() if time.time() - v['start_time'] <= stationary_threshold}

    # Display the frame with bounding boxes
    cv2.imshow('Static Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 31ms/step


In [11]:
import cv2
import numpy as np
import time

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Initialize video capture (0 for default camera, or provide video file path)
cap = cv2.VideoCapture(0)

# Parameters
stationary_threshold = 120  # Time in seconds (2 minutes)
stationary_objects = {}  # Dictionary to track stationary objects

def preprocess_frame(frame):
    """Preprocess frame for model prediction."""
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    return net.forward(output_layers)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame for prediction
    detections = preprocess_frame(frame)

    # Get frame dimensions
    height, width = frame.shape[:2]

    # Process detections
    for out in detections:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Filter for vehicles with confidence threshold
            if class_id in [2, 3, 5, 7]:  # COCO classes for car, motorcycle, bus, and truck
                if confidence > 0.5:
                    # Get bounding box coordinates
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    bbox = (x, y, w, h)

                    # Track stationary objects
                    if class_id not in stationary_objects:
                        stationary_objects[class_id] = {'start_time': time.time(), 'bbox': bbox}
                    else:
                        elapsed_time = time.time() - stationary_objects[class_id]['start_time']
                        if elapsed_time > stationary_threshold:
                            # Draw red bounding box around the vehicle
                            (x, y, w, h) = stationary_objects[class_id]['bbox']
                            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                        else:
                            stationary_objects[class_id]['bbox'] = bbox

    # Remove non-stationary or moved vehicles
    stationary_objects = {k: v for k, v in stationary_objects.items() if time.time() - v['start_time'] <= stationary_threshold}

    # Display the frame with bounding boxes
    cv2.imshow('Static Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'cv::dnn::dnn4_v20231225::readNetFromDarknet'


In [13]:
import cv2
import torch
import time
import numpy as np

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # You can use 'yolov5m', 'yolov5l', 'yolov5x' for larger models

# Initialize video capture (0 for default camera, or provide video file path)
cap = cv2.VideoCapture(0)

# Parameters
stationary_threshold = 10  # Time in seconds (2 minutes)
stationary_objects = {}  # Dictionary to track stationary objects

def detect_vehicles(frame):
    """Detect vehicles using YOLOv5 model."""
    results = model(frame)
    return results.pandas().xyxy[0]  # Extract bounding box data as pandas dataframe

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect vehicles in the frame
    detections = detect_vehicles(frame)

    # Process detections
    for index, row in detections.iterrows():
        class_id = int(row['class'])
        confidence = row['confidence']
        if class_id in [2, 3, 5, 7] and confidence > 0.5:  # Classes: car, motorcycle, bus, truck
            x, y, w, h = int(row['xmin']), int(row['ymin']), int(row['xmax'] - row['xmin']), int(row['ymax'] - row['ymin'])
            bbox = (x, y, w, h)

            # Track stationary objects
            if class_id not in stationary_objects:
                stationary_objects[class_id] = {'start_time': time.time(), 'bbox': bbox}
            else:
                elapsed_time = time.time() - stationary_objects[class_id]['start_time']
                if elapsed_time > stationary_threshold:
                    # Draw red bounding box around the vehicle
                    (x, y, w, h) = stationary_objects[class_id]['bbox']
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                else:
                    stationary_objects[class_id]['bbox'] = bbox

    # Remove non-stationary or moved vehicles
    stationary_objects = {k: v for k, v in stationary_objects.items() if time.time() - v['start_time'] <= stationary_threshold}

    # Display the frame with bounding boxes
    cv2.imshow('Static Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\mahnoor/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-25 Python-3.11.4 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [22]:
import cv2
import torch
import time
import numpy as np

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # You can use 'yolov5m', 'yolov5l', 'yolov5x' for larger models

# Initialize video capture (0 for default camera, or provide video file path)
cap = cv2.VideoCapture(0)

# Parameters
stationary_threshold = 2  # Time in seconds (2 minutes)
stationary_objects = {}  # Dictionary to track stationary objects

def detect_vehicles(frame):
    """Detect vehicles using YOLOv5 model."""
    results = model(frame)
    return results.pandas().xyxy[0]  # Extract bounding box data as pandas dataframe

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect vehicles in the frame
    detections = detect_vehicles(frame)

    # Get frame dimensions
    height, width = frame.shape[:2]

    # Process detections
    for index, row in detections.iterrows():
        class_id = int(row['class'])
        confidence = row['confidence']
        if class_id in [2, 3, 5, 7] and confidence > 0.5:  # Classes: car, motorcycle, bus, truck
            x, y, w, h = int(row['xmin']), int(row['ymin']), int(row['xmax'] - row['xmin']), int(row['ymax'] - row['ymin'])
            bbox = (x, y, w, h)

            # Track stationary objects
            if class_id not in stationary_objects:
                stationary_objects[class_id] = {'start_time': time.time(), 'bbox': bbox, 'detected_time': time.time()}
            else:
                stationary_objects[class_id]['bbox'] = bbox
                stationary_objects[class_id]['detected_time'] = time.time()

    # Draw bounding boxes for all tracked objects
    current_time = time.time()
    for obj_id, obj_data in stationary_objects.items():
        x, y, w, h = obj_data['bbox']
        elapsed_time = current_time - obj_data['detected_time']
        color = (0, 255, 0)  # Green by default
        if elapsed_time > stationary_threshold:
            color = (0, 0, 255)  # Red if stationary for too long
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    # Remove non-stationary or moved vehicles
    stationary_objects = {k: v for k, v in stationary_objects.items() if current_time - v['detected_time'] <= stationary_threshold}

    # Display the frame with bounding boxes
    cv2.imshow('Static Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\mahnoor/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-25 Python-3.11.4 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [21]:
import cv2
import torch
import time
import numpy as np

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # You can use 'yolov5m', 'yolov5l', 'yolov5x' for larger models

# Initialize video capture (0 for default camera, or provide video file path)
cap = cv2.VideoCapture(0)

# Parameters
stationary_threshold = 2  # Time in seconds (2 minutes)
stationary_objects = {}  # Dictionary to track stationary objects

def detect_vehicles(frame):
    """Detect vehicles using YOLOv5 model."""
    results = model(frame)
    return results.pandas().xyxy[0]  # Extract bounding box data as pandas dataframe

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect vehicles in the frame
    detections = detect_vehicles(frame)

    # Get frame dimensions
    height, width = frame.shape[:2]

    # Process detections
    for index, row in detections.iterrows():
        class_id = int(row['class'])
        confidence = row['confidence']
        if class_id in [2, 3, 5, 7] and confidence > 0.5:  # Classes: car, motorcycle, bus, truck
            x, y, w, h = int(row['xmin']), int(row['ymin']), int(row['xmax'] - row['xmin']), int(row['ymax'] - row['ymin'])
            bbox = (x, y, w, h)

            # Track stationary objects
            if class_id not in stationary_objects:
                stationary_objects[class_id] = {'start_time': time.time(), 'bbox': bbox, 'color': (0, 255, 0)}
            else:
                stationary_objects[class_id]['bbox'] = bbox
                if stationary_objects[class_id]['color'] == (0, 255, 0):  # If the box is still green
                    elapsed_time = time.time() - stationary_objects[class_id]['start_time']
                    if elapsed_time > stationary_threshold:
                        stationary_objects[class_id]['color'] = (0, 0, 255)  # Turn red if stationary too long

    # Draw bounding boxes for all tracked objects
    for obj_id, obj_data in stationary_objects.items():
        x, y, w, h = obj_data['bbox']
        color = obj_data['color']
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    # Remove non-stationary or moved vehicles
    stationary_objects = {k: v for k, v in stationary_objects.items() if current_time - v['start_time'] <= stationary_threshold or v['color'] == (0, 0, 255)}

    # Display the frame with bounding boxes
    cv2.imshow('Static Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\mahnoor/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-25 Python-3.11.4 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
import cv2
import torch
import time
import numpy as np

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # You can use 'yolov5m', 'yolov5l', 'yolov5x' for larger models

# Initialize video capture (0 for default camera, or provide video file path)
cap = cv2.VideoCapture(0)

# Parameters
stationary_threshold = 2  # Time in seconds
stationary_objects = {}  # Dictionary to track stationary objects

def detect_vehicles(frame):
    """Detect vehicles using YOLOv5 model."""
    results = model(frame)
    return results.pandas().xyxy[0]  # Extract bounding box data as pandas dataframe

def get_unique_id(bbox, existing_ids):
    """Generate a unique ID for a detected object based on its bounding box."""
    x, y, w, h = bbox
    unique_id = f"{x}_{y}_{w}_{h}"
    while unique_id in existing_ids:
        unique_id += "_1"
    return unique_id

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect vehicles in the frame
    detections = detect_vehicles(frame)

    # Get frame dimensions
    height, width = frame.shape[:2]

    current_time = time.time()

    # List of currently detected objects' IDs
    current_ids = []

    # Process detections
    for index, row in detections.iterrows():
        class_id = int(row['class'])
        confidence = row['confidence']
        if class_id in [2, 3, 5, 7] and confidence > 0.5:  # Classes: car, motorcycle, bus, truck
            x, y, w, h = int(row['xmin']), int(row['ymin']), int(row['xmax'] - row['xmin']), int(row['ymax'] - row['ymin'])
            bbox = (x, y, w, h)

            unique_id = get_unique_id(bbox, stationary_objects.keys())
            current_ids.append(unique_id)

            # Track stationary objects
            if unique_id not in stationary_objects:
                stationary_objects[unique_id] = {'start_time': current_time, 'bbox': bbox, 'color': (0, 255, 0)}
            else:
                stationary_objects[unique_id]['bbox'] = bbox
                elapsed_time = current_time - stationary_objects[unique_id]['start_time']
                if elapsed_time > stationary_threshold:
                    stationary_objects[unique_id]['color'] = (0, 0, 255)  # Turn red if stationary too long

    # Remove objects not detected in the current frame
    stationary_objects = {k: v for k, v in stationary_objects.items() if k in current_ids}

    # Draw bounding boxes for all tracked objects
    for obj_id, obj_data in stationary_objects.items():
        x, y, w, h = obj_data['bbox']
        color = obj_data['color']
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    # Display the frame with bounding boxes
    cv2.imshow('Static Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\mahnoor/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-25 Python-3.11.4 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
